In [1]:
from numpy import genfromtxt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import argparse
import codecs
import numpy as np
import os
import shutil
import torch
import yaml
from datetime import datetime
from torch.utils.data.sampler import RandomSampler
from torch.autograd import Variable


In [2]:
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor([1]))
predictionVar = Variable(torch.LongTensor([[1, 0]]))
print(predictionVar.size())
print(criterion(predictionVar, trainLabelVar))


torch.Size([1, 2])


RuntimeError: nll_loss_forward is not implemented for type torch.LongTensor

In [ ]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [ ]:
train = data.iloc[0:1611]
test = data[1611:]
print(train.shape)
print(test.shape)

In [ ]:
trainheadlines = data.iloc[0:1611, 2:27].values
trainLabels = data.iloc[0:1611, 1].values
testheadlines = data.iloc[1611:, 2:27].values
testLabels = data.iloc[1611:, 1].values

print(trainheadlines.shape)
print(trainLabels[:10])

In [ ]:
trainheadlines = []
trainLabels = []
for row in range(0,len(train.index)):
    trainheadlines += [str(x) for x in train.iloc[row,2:27]]
    trainLabels.append(int(train.iloc[row,1]))

basicvectorizer = CountVectorizer(min_df = 5)
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)
print(len(trainLabels))

testheadlines = []
testLabels = []
for row in range(0,len(test.index)):
    testheadlines += [str(x) for x in test.iloc[row,2:27]]
    testLabels.append(test.iloc[row,1])

basictest = basicvectorizer.transform(testheadlines)
print(basictest.shape)


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import tensorflow as tf

class LSTMLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LSTMLayer, self).__init__()
        self.output_dim = output_dim
        self.lstmLayers = []
        self.basicLSTM = nn.LSTM(input_dim, output_dim, batch_first = True)
        self.fc = nn.Linear(output_dim, 2)
        self.maxPool = nn.MaxPool1d(25)

    def forward(self, headlines, hidden = None, cell = None):
        outputs = []
        new_hiddens = []
        if hidden is None:
            hidden = Variable(torch.zeros(1, 1, self.output_dim)).float()   #1x1x30
            cell = Variable(torch.zeros(1, 1, self.output_dim)).float()     #1x1x30
        headlinesTransformed = Variable(torch.from_numpy(np.asarray(headlines.todense().astype(float)))).view(1,25,-1).float()
        if torch.cuda.is_available():
            hidden = hidden.cuda()
            cell = cell.cuda()
            headlinesTransformed = headlinesTransformed.cuda()
#         print(headlines.todense().shape)
            headlinesTransformed = headlinesTransformed.cuda();
        output, (hidden, cell) = self.basicLSTM(headlinesTransformed, (hidden, cell))
        permutedOutput = output.permute(0, 2, 1)
        maxPooledOutput = self.maxPool(permutedOutput).view(1, 1, -1)
#         print(maxPooledOutput.size())
        log_probs = F.log_softmax(self.fc(maxPooledOutput[0]), dim=-1)
        return log_probs, hidden, cell


In [7]:
lstmLayer = LSTMLayer(basictrain[0].shape[1], 30).cuda()
learning_rate = 0.3
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=learning_rate)
epoch = 0
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
train_loss = []
criterion = torch.nn.NLLLoss()
trainLabelVar = Variable(torch.LongTensor(trainLabels)).cuda()

while epoch < 100:
    cell = None
    hidden = None
    total_loss = 0
    for i in range(len(trainLabels)):
#         if i == 100:
#             break
        lstmOptimizer.zero_grad()
        log_probs, hidden, cell = lstmLayer(basictrain[i*25: (i+1)*25], hidden, cell)
        loss = criterion(log_probs, trainLabelVar[i])
        total_loss += loss
    total_loss.backward()
        
    lstmOptimizer.step()
    train_loss.append(total_loss.data)
    if ((epoch) % 10) == 0:
        print(train_log_string % (datetime.now(), epoch, i+1, len(trainheadlines), np.mean(train_loss)))
#         print('')
    epoch = epoch + 1

NameError: name 'lstmOptimizer' is not defined

In [ ]:
state_dict = {
    'epoch': epoch,
    'lstmLayer': lstmLayer.state_dict(),
    'lstm_optimizer': lstmOptimizer.state_dict()
}
ckpt_path = 'data\ckpt\basic_most\model.pt'
# torch.save(state_dict, '/data/ckpt/basic_most/model.pt')
torch.save(state_dict, 'bow_25_model_full_epoch200_mindf_5.pt')

In [8]:
# del lstmLayer
# del lstmOptimizer
# del train_loss
lstmLayer = LSTMLayer(basictrain[0].shape[1], 30).cuda()
lstmOptimizer = torch.optim.Adam(lstmLayer.parameters(), lr=0.3)
ckpt = torch.load('bow_25_model_full_epoch200_mindf_5.pt')
lstmLayer.load_state_dict(ckpt['lstmLayer'])
lstmOptimizer.load_state_dict(ckpt['lstm_optimizer'])

In [9]:
# lstmLayer.cpu()
trainPredictions = []
log_probs = []
h = None
c = None
for i in range(len(trainLabels)):
    log_prob, h, c = lstmLayer(basictrain[i*25: (i+1)*25], h, c)
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > float(log_prob[0][1]):
        trainPredictions.append(0)
    else:
        trainPredictions.append(1)

In [ ]:
print('Training Accuracy:', sum(np.asarray(trainPredictions) == np.asarray(trainLabels))/len(trainLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(trainLabels, trainPredictions))

In [ ]:
del lstmOptimizer
del log_probs

In [27]:

testPredictions = []
log_probs = []
# h = None
# c = None
for i in range(len(testLabels)):
    log_prob, h, c = lstmLayer(basictest[i*25: (i+1)*25], h, c)
    log_probs.append(log_prob)
    if float(log_prob[0][0]) > 1.2*float(log_prob[0][1]):
        testPredictions.append(0)
    else:
        testPredictions.append(1)

AttributeError: 'int' object has no attribute 'cuda'

In [11]:
print('Test Accuracy:', sum(np.asarray(testPredictions) == np.asarray(testLabels))/len(testLabels))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(testLabels, testPredictions))

Test Accuracy: 0.52380952381
[[ 54 132]
 [ 48 144]]


In [ ]:
sum(sum(confusion_matrix(testLabels, testPredictions)))

In [ ]:
for i in range(len(testLabels)):
    if testLabels[i] != testPredictions[i]:
        print(log_probs[i], testLabels[i], testPredictions[i])

In [22]:
back = log_probs

In [20]:
c = 0
for i in range(len(testLabels)):
    if float(log_probs[i][0][0]) > 0:
        c += 1

In [26]:
torch.cuda.is_available()

True